# Code Example

** THIS IS JUST A PLACE HOLDER PLEASE IGNORE ** 


## Digital Circuits

In [1]:
import numpy as np

from qilisdk.digital import CNOT, RX, Circuit, H, M
from qilisdk.backends import QutipBackend
from qilisdk.functionals import Sampling

# Create a circuit with 2 qubits
circuit = Circuit(2)
circuit.add(H(0))  # Apply Hadamard on qubit 0
circuit.add(RX(0, theta=np.pi))  # Apply RX rotation on qubit 0con
circuit.add(CNOT(0, 1))  # Add a CNOT gate between qubit 0 and 1

# Retrieve the current gate parameters
print("Initial parameters:", circuit.get_parameter_values())

# Update circuit parameters (e.g., update RX rotation angle)
circuit.set_parameter_values([2 * np.pi])

# Execute the circuit simulation using QutipBackend
backend = QutipBackend()
results = backend.execute(Sampling(circuit))

# Display the simulation output and measurement probabilities
print("Simulation Results:")
print(results)
print("Probabilities:", results.probabilities)

Initial parameters: [3.141592653589793]
Simulation Results:
SamplingResult(
  nshots=1000,
  samples={'00': 517, '11': 483}
)
Probabilities: {'11': 0.483, '00': 0.517}


## Hamiltonians

In [2]:
from qilisdk.analog import Hamiltonian, X, Z

# Build a Hamiltonian expression using Pauli operators
H_expr = (Z(0) + X(0)) * (Z(0) - X(0))
print("Hamiltonian Expression:", H_expr)

# Iterate over Hamiltonian terms
for coeff, ops in H_expr:
    print("Coefficient:", coeff, "Operators:", ops)

# Parse a Hamiltonian from its string representation
parsed_H = Hamiltonian.parse("-2j Y(0)")
print(H_expr == parsed_H)

Hamiltonian Expression: -2j Y(0)
Coefficient: -2j Operators: [Y(0)]
True


In [3]:
H = Z(0) + 2 * X(1) * Z(1) + 3 * Z(1) + 2 * Z(0)

print(H)
for coeff, op in H:
    print(coeff, op)

3 Z(0) - 2j Y(1) + 3 Z(1)
(3+0j) [Z(0)]
-2j [Y(1)]
(3+0j) [Z(1)]


## Analog Time-Evolution Simulation

In [4]:
import numpy as np

from qilisdk.analog import Schedule, X, Y, Z
from qilisdk.backends import QutipBackend
from qilisdk.common import ket, tensor_prod
from qilisdk.functionals import TimeEvolution

T = 10  # Total evolution time
dt = 0.1  # Time-step
steps = np.linspace(0, T, int(T / dt))
nqubits = 1

# Define two Hamiltonians for the simulation
H1 = sum(X(i) for i in range(nqubits))
H2 = sum(Z(i) for i in range(nqubits))

# Create a schedule for the time evolution
schedule = Schedule(
    T,
    dt,
    hamiltonians={"h1": H1, "h2": H2},
    schedule={
        t: {
            "h1": 1 - steps[t] / T,
            "h2": steps[t] / T
            } for t in range(len(steps))
        },
)

# Prepare an initial state (equal superposition)
state = tensor_prod([(ket(0) + ket(1)).unit() for _ in range(nqubits)]).unit()

# Perform time evolution on the CUDA backend with observables to monitor
time_evolution = TimeEvolution(
    schedule=schedule,
    initial_state=state,
    observables=[Z(0), X(0), Y(0)],
    store_intermediate_results=True
)

backend = QutipBackend()
results = backend.execute(time_evolution)
print("Time Evolution Results:", results)

Time Evolution Results: TimeEvolutionResult(
  final_expected_values=array([ 0.99426886, -0.01322734,  0.10608707]),
  expected_values=array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.19420124e-09,  1.00000000e+00,  2.43282688e-06],
       [ 1.97549858e-04,  9.99997984e-01,  1.99816005e-03],
       [ 9.76438099e-04,  9.99981916e-01,  5.93418774e-03],
       [ 2.68656871e-03,  9.99928468e-01,  1.16550558e-02],
       [ 5.63154892e-03,  9.99804577e-01,  1.89497368e-02],
       [ 1.00618836e-02,  9.99568682e-01,  2.75900245e-02],
       [ 1.61724846e-02,  9.99174172e-01,  3.72749836e-02],
       [ 2.40908416e-02,  9.98571728e-01,  4.76879030e-02],
       [ 3.38808586e-02,  9.97711109e-01,  5.85203383e-02],
       [ 4.55466009e-02,  9.96544807e-01,  6.94546953e-02],
       [ 5.90360431e-02,  9.95029771e-01,  8.01903972e-02],
       [ 7.42449136e-02,  9.93128240e-01,  9.04654192e-02],
       [ 9.10273659e-02,  9.90814072e-01,  1.00007469e-01],
       [ 1.09195789e-01, 

## Building Models

In [5]:
from qilisdk.common.variables import BinaryVariable, Bitwise, Domain, SpinVariable, Variable

x = Variable("x", domain=Domain.REAL, bounds=(1, 2), encoding=Bitwise, precision=1e-1)
s = SpinVariable("s")
b = BinaryVariable("b")

In [6]:
x.to_binary()

(0.1) * x(0) + (0.2) * x(1) + (0.4) * x(2) + (0.30000000000000004) * x(3) + (1.0)

In [7]:
x.evaluate([0, 1, 0, 0])

1.2

Examples of building Terms from variables:

In [8]:
t1 = 2 * x + 3
print("t1: ", t1)
t2 = 3 * x**2 + 2 * x + 4
print("t2: ", t2)
t3 = 2 * x + b - 1
print("t3: ", t3)
t4 = t1 - t2
print("t4: ", t4)

t1:  (2) * x + (3)
t2:  (3) * (x^2) + (2) * x + (4)
t3:  (2) * x + b + (-1)
t4:  (-1.0) + (-3.0) * (x^2)


In [9]:
t3.evaluate({
    x: 1.5,
    b: 0
})

2.0

In [10]:
from qilisdk.common.variables import LT

LT(2 * x - 1, 1)

(2) * x < (2.0)

In [11]:
from qilisdk.common.model import Model, ObjectiveSense

model = Model("example_model")

model.set_objective(2*x + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 100 
	 x_lower_bound_constraint : 100 



In [12]:
model.set_lagrange_multiplier("x_upper_bound_constraint", 1)

print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 1 
	 x_lower_bound_constraint : 100 



In [13]:
model.add_constraint("test_constraint", LT(x, 1.5), lagrange_multiplier=10)
print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the constraint/s: 
	 test_constraint: x < (1.5) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 1 
	 x_lower_bound_constraint : 100 
	 test_constraint : 10 



In [14]:
model.evaluate({
    x: 1.4
})

{'obj': 5.8, 'test_constraint': 0.0}

In [15]:
model.evaluate({x: 2})

{'obj': 7.0, 'test_constraint': 10.0}

In [16]:

from qilisdk.common.model import QUBO, ObjectiveSense

model = QUBO("example_qubo_model")
model.set_objective(2 * b + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

model.add_constraint("qubo_constraint", LT(2 * b, 1), lagrange_multiplier=10, penalization="slack", transform_to_qubo=True)

print(model)

Model name: example_qubo_model 
objective (obj): 
	 minimize : 
	 (2) * b + (3) 

subject to the constraint/s: 
	 qubo_constraint: (4.0) * (b * qubo_constraint_slack(0)) + (-1.0) * qubo_constraint_slack(0) == (-1.0) 

With Lagrange Multiplier/s: 
	 qubo_constraint : 10 



In [17]:
print(model.qubo_objective)

obj: (2) * b + (13.0) + (40.0) * (b * qubo_constraint_slack(0)) + (-10.0) * qubo_constraint_slack(0)


In [18]:
from qilisdk.common.model import QUBO, ObjectiveSense

model = QUBO("example_qubo_model")
model.set_objective(2 * b + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

model.add_constraint(
    "qubo_constraint", LT(2 * b, 1), lagrange_multiplier=1, penalization="unbalanced", parameters=[1, 2], transform_to_qubo=True
)

print(model)

Model name: example_qubo_model 
objective (obj): 
	 minimize : 
	 (2) * b + (3) 

subject to the constraint/s: 
	 qubo_constraint: (2.0) * b == (-1.0) 

With Lagrange Multiplier/s: 
	 qubo_constraint : 1 



In [23]:
model.qubo_objective

obj: (4.0) * b + (4.0)

## Quantum Objects

In [ ]:
from qilisdk.common.quantum_objects import QuantumObject

psi = QuantumObject(np.array(
    [[1],
     [0]]
    ))
print(psi, psi.is_ket())
print("-" * 10)

psi = QuantumObject(np.array([1, 0]))
print(psi, psi.is_bra())
print("-" * 10)

psi = QuantumObject(np.array([[1, 0], [0, 0]]))
print(psi, psi.is_density_matrix())
print("-" * 10)

[[1]
 [0]] True
----------
[[1 0]] True
----------
[[1 0]
 [0 0]] True
----------


In [ ]:
from qilisdk.common.quantum_objects import ket, bra, basis_state

psi = ket(0)
print(psi, psi.is_ket())
print("-" * 10)

In [20]:
from qilisdk.common.quantum_objects import QuantumObject, basis_state, ket, bra, tensor_prod, expect_val
import numpy as np

# 1) Create a Bell state |Φ+> = (|00> + |11>)/√2
zero = basis_state(0, 2)
one = basis_state(1, 2)
bell = (tensor_prod([zero, zero]) + tensor_prod([one, one])).unit()

# 2) Verify partial trace yields maximally mixed one‑qubit state
rho_bell = bell.to_density_matrix()
rho_A = rho_bell.ptrace([0])  # trace out qubit B
print("rho_A trace:", rho_A.dense.trace())  # should ≈ 1
print("rho_A equals I/2?", rho_A.dense)  # ≈ [[0.5,0],[0,0.5]]

# 3) Compute expectation of Z⊗Z on the Bell state
Z = QuantumObject(np.array([[1, 0], [0, -1]]))
ZZ = tensor_prod([Z, Z])
ev_ZZ = expect_val(ZZ, rho_bell)
print("⟨ZZ⟩:", ev_ZZ)  # should be +1

rho_A trace: 1.0
rho_A equals I/2? [[0.5 0. ]
 [0.  0.5]]
⟨ZZ⟩: 1.0


## Exporting and Importing Circuits

In [21]:
import numpy as np

from qilisdk.digital import CNOT, RX, Circuit, H
from qilisdk.utils.openqasm2 import from_qasm2, from_qasm2_file, to_qasm2, to_qasm2_file

# Create a sample circuit
circuit = Circuit(3)
circuit.add(H(0))
circuit.add(CNOT(0, 1))
circuit.add(RX(2, theta=np.pi))
circuit.add(M(0, 1, 2))

# Serialize to QASM string
qasm_code = to_qasm2(circuit)
print("Generated QASM:")
print(qasm_code)

# Deserialize back to a circuit
reconstructed_circuit = from_qasm2(qasm_code)

# Save to and load from a file
to_qasm2_file(circuit, "circuit.qasm")
reconstructed_circuit = from_qasm2_file("circuit.qasm")

Generated QASM:
OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
creg c[3];
h q[0];
cx q[0], q[1];
rx(3.141592653589793) q[2];
measure q -> c;


## Serialization

In [22]:
from qilisdk.digital import CNOT, Circuit, H, M
from qilisdk.utils.serialization import deserialize, deserialize_from, serialize, serialize_to

circuit = Circuit(2)
circuit.add(H(0))
circuit.add(CNOT(0, 1))
circuit.add(M(0, 1))

# Serialize a circuit to a YAML string
yaml_string = serialize(circuit)

# Deserialize back into a Circuit object
restored_circuit = deserialize(yaml_string, cls=Circuit)

# Alternatively, work with files:
serialize_to(circuit, 'circuit.yml')
restored_circuit = deserialize_from('circuit.yml', cls=Circuit)